In [1]:
import pandas as pd
from model_utils import *
import requests
import json

df_resumes = pd.read_csv('data/ResumeDataSet.csv')
df_jobs =  pd.read_csv('data/li_jobs.zip')

In [12]:
resume_emb = get_dataset_embeddings(df_resumes, columns=['Category', 'Resume'], weights= [0.5, 0.5])
job_emb = get_dataset_embeddings(df_jobs, columns=['title', 'description'], weights= [0.5, 0.5])

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [5]:
data_for_api = []
dfs = df_jobs.sample(1000)
for i, row in dfs.iterrows():
    data_for_api.append({'title': row.title, 'description': row.description, 'skills': row.description})


### Для рботы дальнекйшего блока нужно запустить model_api.py

## CUDA tests

In [13]:
%%time
answers = []
url = 'http://127.0.0.1:8000/single_embeddings'
for data in data_for_api:

    x = requests.post(url, json=data)
    dec = np.asarray(json.loads(x.json()['emb']))
    answers.append(dec)
    

CPU times: total: 1.86 s
Wall time: 31.7 s


In [20]:
%%time
titles = []
desc = []
skills = []
for data in data_for_api:
    titles.append(data['title'])
    desc.append(data['description'])
    skills.append(data['skills'])

url = 'http://127.0.0.1:8000/batch_embeddings'

myobj = {'title': titles,
         'description': desc,
         'skills': skills}

x = requests.post(url, json=myobj)
dec = np.asarray([json.loads(X) for X in x.json()['emb']])


CPU times: total: 250 ms
Wall time: 9.7 s


## CPU tests

In [22]:
%%time
answers = []
url = 'http://127.0.0.1:8000/single_embeddings'
for data in data_for_api:

    x = requests.post(url, json=data)
    dec = np.asarray(json.loads(x.json()['emb']))
    answers.append(dec)
    

CPU times: total: 2.02 s
Wall time: 4min 10s


In [23]:
%%time
titles = []
desc = []
skills = []
for data in data_for_api:
    titles.append(data['title'])
    desc.append(data['description'])
    skills.append(data['skills'])

url = 'http://127.0.0.1:8000/batch_embeddings'

myobj = {'title': titles,
         'description': desc,
         'skills': skills}

x = requests.post(url, json=myobj)
dec = np.asarray([json.loads(X) for X in x.json()['emb']])


CPU times: total: 234 ms
Wall time: 4min 23s


### Подбор вакансий для резюме из датасета

In [13]:
get_rec_from_dataset(df_resume = df_resumes, df_jobs = df_jobs, prompt_embeddings=resume_emb, jobs_embeddings=job_emb, num_rec=5)

Areas of Interest Deep Learning, Control System Design, Programming in-Python, Electric Machinery, Web Development, Analytics Technical Activities q Hindustan Aeronautics Limited, Bangalore - For 4 weeks under the guidance of Mr. Satish, Senior Engineer in the hangar of Mirage 2000 fighter aircraft Technical Skills Programming Matlab, Python and Java, LabView, Python WebFrameWork-Django, Flask, LTSPICE-intermediate Languages and and MIPOWER-intermediate, Github (GitBash), Jupyter Notebook, Xampp, MySQL-Basics, Python Software Packages Interpreters-Anaconda, Python2, Python3, Pycharm, Java IDE-Eclipse Operating Systems Windows, Ubuntu, Debian-Kali Linux Education Details 
January 2019 B.Tech. Electrical and Electronics Engineering  Manipal Institute of Technology
January 2015    DEEKSHA CENTER
January 2013    Little Flower Public School
August 2000    Manipal Academy of Higher
DATA SCIENCE 

DATA SCIENCE AND ELECTRICAL ENTHUSIAST
Skill Details 
Data Analysis- Exprience - Less than 1 yea

,Unnamed: 0,id,created,last_updated,time_posted,title,description,seniority,employment_type,location,url
7560,7560,172132931,2023-08-04 10:25:56,2023-08-30 05:46:42,3 weeks ago,Data Analyst,• Interesting technical work|Strong compensati...,Entry level,Full-time,"Toronto, Ontario, Canada",https://www.linkedin.com/jobs/view/data-analys...
9521,9521,171211953,2023-07-31 17:52:03,2023-08-10 21:28:18,3 days ago,Data Engineer,Stefanini is looking for a Data Engineer in Da...,Mid-Senior level,Contract,"Dallas, TX",https://www.linkedin.com/jobs/view/data-engine...
9899,9899,177466573,2023-08-25 14:54:44,2023-08-30 23:49:35,1 month ago,Data Scientist,CFD Research in looking for someone that will ...,Entry level,Full-time,"Huntsville, AL",https://www.linkedin.com/jobs/view/data-scient...
2220,2220,146292526,2023-05-03 15:52:48,2023-08-05 09:09:43,2 months ago,Data Scientist,"About Cubist\n\nCubist Systematic Strategies, ...",Entry level,Full-time,"New York, United States",https://www.linkedin.com/jobs/view/data-scient...
5417,5417,169603791,2023-07-26 06:09:57,2023-08-29 12:57:36,1 month ago,Data Engineer,"Bachelors degree in Computer Science, or 10+ c...",Entry level,Contract,"Atlanta, GA",https://www.linkedin.com/jobs/view/data-engine...


### Подбор вакансий для описания

In [14]:
job_title = 'Junior Data Engineer'

job_text = """
Rewrite following for job resume:
Comprehensive analysis of customer experience in the ATM network through:
- exploring data sources and search for insights;
- data cleaning and processing;
- development and deployment of datamarts;
- calculation and control of product metrics;
- collection and formalization of business requirements;
- dashboards creation.

"""



job_title = 'Master Electrician'

job_text = """
    Technical-minded electrician with XX years of experience installing, repairing, testing, and maintaining electrical systems, wiring, equipment, appliances, and fixtures. Skilled in diagnosing and replacing high-voltage electrical power lines, pole wiring, and underground lines. Adept in managing service calls, troubleshooting electrical systems, and resolving issues with equipment.Skilled in diagnosing and replacing high voltage electrical power lines, pole wiring, and underground lines; installing circuit breakers, transformers, stabilizers, generators, and distribution systems; capable of quickly understanding blueprints, schematics, and technical manuals
    Adept in managing service calls, troubleshooting electrical systems, and resolving issues with equipment; setting-up home and business backup generators with auto transfer switches to ensure minimal downtime in a power outage; directing and training workers to install, maintain, or repair electrical wiring, equipment, and fixtures
"""


get_rec_from_scratch([job_title, job_text], weights=[0.5, 0.5], df_jobs=df_jobs, jobs_embeddings=job_emb, num_rec = 5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Max scores: [[[0.6559852  0.6482438  0.6446736  0.6446736  0.63523066]]]


,Unnamed: 0,id,created,last_updated,time_posted,title,description,seniority,employment_type,location,url
190,190,144187986,2023-04-26 01:58:03,2023-06-09 22:17:10,1 month ago,Commercial Electrician,About Us\n\nOwen Electric Company has been in ...,Entry level,Full-time,"Jacksonville, FL",https://www.linkedin.com/jobs/view/commercial-...
3385,3385,169011373,2023-07-24 21:13:48,2023-08-29 12:58:48,2 months ago,Electrician,"SIGN ON BONUS UPTO $2,000.00\n\nThe Romanoff G...",Entry level,Full-time,"Dayton, OH",https://www.linkedin.com/jobs/view/electrician...
8584,8584,154049609,2023-05-26 13:52:46,2023-06-04 20:27:22,1 hour ago,Licensed Electrician,"Faith Technologies, a division of Faith Techno...",Entry level,Internship,"Pasco, WA",https://www.linkedin.com/jobs/view/licensed-el...
8416,8416,150815455,2023-05-15 12:51:25,2023-06-18 04:37:36,1 month ago,Licensed Electrician,"Faith Technologies, a division of Faith Techno...",Entry level,Internship,"Kennewick, WA",https://www.linkedin.com/jobs/view/licensed-el...
9664,9664,173693904,2023-08-09 12:52:14,2023-08-29 19:34:06,1 month ago,Service & Repair Electrician,Join Our Team of Professional Electricians\n\n...,Entry level,Full-time,"Pittsburgh, PA",https://www.linkedin.com/jobs/view/service-rep...


In [14]:
df_jobs =  pd.read_csv('data/li_jobs.zip')
df_resumes = pd.read_csv('data/ResumeDataSet.csv')

# TFidf vec

In [15]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vyacheslav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [16]:
corpus = df_jobs.description.values

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                             token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                             stop_words=stop_words)

vectorizer1.fit(corpus)

vocab_df_valid = vectorizer1.get_feature_names_out()

C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'"] not in stop_words.
  warnings.warn(


In [18]:
vocab_df_valid

array(['!', '! !', '! "', ..., '𝐳𝐨𝐧𝐞𝐬 process', '𝗦𝗸𝗶𝗹𝗹𝘀',
       '𝗦𝗸𝗶𝗹𝗹𝘀 business'], dtype=object)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                             token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                             stop_words=stop_words)

vectorizer1.fit(corpus)

vocab_df_valid = vectorizer1.get_feature_names_out()


vectorizer = TfidfVectorizer(ngram_range=(1,2),
                             token_pattern=r"(?u)\b[\w/.-]+\b|!|/|\?|\"|\'",
                             stop_words=stop_words,
                             vocabulary=vocab_df_valid)
vectorizer.fit(corpus)
corpus_tf_idf = vectorizer.transform(corpus)

print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")


C:\Users\Vyacheslav\PycharmProjects\CV_projects\venv\lib\site-packages\sklearn\feature_extraction\text.py:1369: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


length of vectorizer vocab is 1027539


In [20]:
resume_tf_idf = vectorizer.transform(df_resumes.Resume)

In [21]:
corpus_tf_idf

<10000x1027539 sparse matrix of type '<class 'numpy.float64'>'
	with 5628178 stored elements in Compressed Sparse Row format>

In [22]:
resume_tf_idf @ corpus_tf_idf.T

<962x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 9456564 stored elements in Compressed Sparse Row format>

In [23]:
import unicodedata

In [24]:
df_resumes.Resume

0      Skills * Programming Languages: Python (pandas...
1      Education Details \r\nMay 2013 to May 2017 B.E...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details \r\n MCA   YMCAUST,  Faridab...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

In [39]:
import re
df_resumes.Resume.map(lambda example: re.sub('[\W]+', ' ', example))
                                             

0      Skills Programming Languages Python pandas num...
1      Education Details May 2013 to May 2017 B E UIT...
2      Areas of Interest Deep Learning Control System...
3      Skills â R â Python â SAP HANA â Tableau â SAP...
4      Education Details MCA YMCAUST Faridabad Haryan...
                             ...                        
957    Computer Skills â Proficient in MS office Word...
958    â Willingness to accept the challenges â Posit...
959    PERSONAL SKILLS â Quick learner â Eagerness to...
960    COMPUTER SKILLS SOFTWARE KNOWLEDGE MS Power Po...
961    Skill Set OS Windows XP 7 8 8 1 10 Database MY...
Name: Resume, Length: 962, dtype: object

In [ ]:
re s